In [28]:
x = {"a":[1,2,3,4,5], "b":[10,20,30,40,50], "c":[100,200,300,400,500],
    "d": [11,22,33,44,55], "e":[111,222,333,444,555]}

In [4]:
x


{'a': [1, 2, 3, 4, 5],
 'b': [10, 20, 30, 40, 50],
 'c': [100, 200, 300, 400, 500]}

In [6]:
import pandas as pd


In [29]:
df = pd.DataFrame(x)
df

,a,b,c,d,e
0,1,10,100,11,111
1,2,20,200,22,222
2,3,30,300,33,333
3,4,40,400,44,444
4,5,50,500,55,555


In [41]:
df.loc[[1,3],["a","c","d"]]

,a,c,d
1,2,200,22
3,4,400,44


In [1]:
df.iloc[0:3,:3]

NameError: name 'df' is not defined

In [4]:
l = [0, -11, 31, 22,  -11, 33, -44, -55]

for i in l:
    if i < 0:
        l.remove(i)
print(l)

[0, 31, 22, 33, -55]


In [8]:
k = ["apple", "dell", "lg", "samsung"]

set(k)




{'apple', 'dell', 'lg', 'samsung'}

In [40]:
k = ["apple", "dell", "samsung", "lg"]
k = set(k)
k
list(k)

['samsung', 'dell', 'lg', 'apple']

In [ ]:
def solution(lottos, win_nums):
    cnt = len(set(lottos) & set(win_nums))
    zero = lottos.count(0)

    return [7-max(cnt+zero,1) ,7-max(cnt,1)]

In [7]:
def solution(numbers, hand):
    # [1, 3, 4, 5, 8, 2, 1, 4, 5, 9, 5]
    import pandas as pd
    key = pd.DataFrame([1,2,3,4,5,6,7,8,9,"*",0,"#"], columns)
    
    answer = ''
    return answer

{0, 1, 6, 20, 30}

In [9]:
import pandas as pd
import numpy as np

In [17]:
key = pd.DataFrame([[1,2,3],[4,5,6],[7,8,9],["*",0,"#"]])
key.columns

RangeIndex(start=0, stop=3, step=1)

In [5]:
def solution(array, commands):
    answer = []
    for c in commands:
        answer.append(array[c[0]-1:c[1]-1].sort()[c[2]-1])
    
    return answer

In [46]:
def solution(array, commands):
    answer = list()
    for c in commands:
    

        
        lst = array[c[0]-1 : c[1]].sort()
        
        answer.append( lst[ c[2] - 1] )
                      
                      
    return answer

In [48]:
answer = list()
array = [8,1,26,4,8,4,5,2]
c = [2,3,4]
        
lst = array[c[0]-1 : c[1]].sort()
        
answer.append( lst[ c[2] - 1] )

TypeError: 'NoneType' object is not subscriptable

In [63]:
answer.append(1)
answer

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [1]:
array = [8,1,26,4,8,4,5,2]
for i in array.index:
    print(i)

TypeError: 'builtin_function_or_method' object is not iterable

In [6]:
d = ["SUN","MON","TUE","WED","THU","FRI","SAT"]
for i in range(a):
    
    

'FRI'

In [58]:
def solution(n):
    
    root = n**(0.5)
    if (root % 1) == 0:
        return (n+1)**2
    else:
        
        return -1
    
solution(121)

14884

In [104]:
n = 121
root = n**(0.5)

if(root % 1) == 0:

    print((121+1)**2)



14884


In [119]:
import numpy as np
np.square(122)

14884